In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy, resample_apply
from backtesting.test import SMA
from backtesting import Backtest

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd 
import math as m
import os, glob

/Users/marami3/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:44: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
PRED = True
WithClassOne = False
CASH = 1000
COMMISSION = 0.0001

In [4]:
pwd = os.getcwd()
datasource_path = os.path.join(pwd, r'../../../Results/LSTM/')
filenames = glob.glob(datasource_path + "/*.csv")
filenames.sort()
for file in filenames:
    print(os.path.basename(file))

5C-LSTM_1H_42F_Class_01_Test.csv
5C-LSTM_1H_42F_Class_01_Train.csv
5C_1D_45F_5137Size_LSTM_10Range_5Epochs_Class_02_Test.csv
5C_1D_45F_5137Size_LSTM_10Range_5Epochs_Class_02_Train.csv
C-LSTM_1D_19F_Class_03_Test.csv
C-LSTM_1D_19F_Class_03_Train.csv
C-LSTM_1H_0F_Class_Test_Results_01.csv
C-LSTM_1H_0F_Class_Test_Results_02.csv
C-LSTM_1H_0F_Class_Train_Results_01.csv
C-LSTM_1H_0F_Class_Train_Results_02.csv


In [15]:
index = 2
NumCLASS =  5


dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
DATASET = pd.read_csv(filenames[index], header=0)
MODEL_NAME = os.path.basename(filenames[index])
print(MODEL_NAME)
print(DATASET.head(20))
print(DATASET.shape)


5C_1D_45F_5137Size_LSTM_10Range_5Epochs_Class_02_Test.csv
       Open     High      Low    Close       Volume  Real_Class  \
0   1.17145  1.17390  1.16697  1.17336  209621.9375         1.0   
1   1.17319  1.17438  1.17308  1.17427    8920.6504         0.0   
2   1.17427  1.17562  1.17198  1.17414  152661.7344         2.0   
3   1.17412  1.18259  1.17408  1.18138  214738.0938         2.0   
4   1.18135  1.18694  1.17955  1.18644  216994.3750        -1.0   
5   1.18641  1.18799  1.18254  1.18314  204817.6406        -1.0   
6   1.18314  1.18750  1.18055  1.18223  199182.5000         0.0   
7   1.18150  1.18183  1.18059  1.18149    7389.2700        -1.0   
8   1.18149  1.18199  1.17808  1.17948  187651.9063        -1.0   
9   1.17946  1.17953  1.17362  1.17716  192445.1250         1.0   
10  1.17712  1.18056  1.17304  1.17946  195770.4844         2.0   
11  1.17945  1.18581  1.17682  1.18488  223686.5156        -2.0   
12  1.18488  1.18582  1.17627  1.17825  207020.6719        -1.0   
13  

In [16]:
if NumCLASS == 3:
    if (PRED):
        DATASET['Position'] = DATASET['Predicted_Class']
        DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = -1 
        DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 1 
    else:
        DATASET['Position'] = DATASET['Real_Class']
        DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = -1
        DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 1 
if NumCLASS == 5:
    if (PRED):
        DATASET['Position'] = DATASET['Predicted_Class']
        if (WithClassOne):
            DATASET.loc[DATASET.Predicted_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Predicted_Class == 2, 'Position'] = 1
        else:
            DATASET.loc[DATASET.Predicted_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Predicted_Class == 2, 'Position'] = 1
            DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = 0 # remove Small movements
            DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 0 #Remove Small movements
    else:
        DATASET['Position'] = DATASET['Real_Class']
        if (WithClassOne):
            DATASET.loc[DATASET.Real_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Real_Class == 2, 'Position'] = 1
        else:
            DATASET.loc[DATASET.Real_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Real_Class == 2, 'Position'] = 1
            DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = 0 # remove Small movements
            DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 0 #Remove Small movements
DATASET.head(10)

,Open,High,Low,Close,Volume,Real_Class,Predicted_Class,Position
0,1.17145,1.17390,1.16697,1.17336,209621.9375,1.0,-2.0,-1.0
1,1.17319,1.17438,1.17308,1.17427,8920.6504,0.0,-2.0,-1.0
2,1.17427,1.17562,1.17198,1.17414,152661.7344,2.0,-2.0,-1.0
3,1.17412,1.18259,1.17408,1.18138,214738.0938,2.0,-2.0,-1.0
4,1.18135,1.18694,1.17955,1.18644,216994.3750,-1.0,2.0,1.0
5,1.18641,1.18799,1.18254,1.18314,204817.6406,-1.0,2.0,1.0
6,1.18314,1.18750,1.18055,1.18223,199182.5000,0.0,2.0,1.0
7,1.18150,1.18183,1.18059,1.18149,7389.2700,-1.0,2.0,1.0
8,1.18149,1.18199,1.17808,1.17948,187651.9063,-1.0,2.0,1.0
9,1.17946,1.17953,1.17362,1.17716,192445.1250,1.0,2.0,1.0


In [17]:
class limits(SignalStrategy, TrailingStrategy):
    buy_lim = 0.99
    sell_lim = -0.99
    
    def init(self):
        super().init()
        self.Op_Abierta = False
        
    def next(self):
        sig = self.data.Position[-1]
        if not self.Op_Abierta and sig >= self.buy_lim:
            self.buy()
            self.Op_Abierta = True
        if self.Op_Abierta and sig <= self.sell_lim:
            self.sell()
            self.Op_Abierta = False

In [18]:
class limits2(SignalStrategy, TrailingStrategy):
    buy_lim = 0.99
    sell_lim = -0.99
    
    def init(self):
        super().init()
        
    def next(self):
        sig = self.data.Position[-1]
        if  sig >= self.buy_lim:
            self.buy()
        if  sig <= self.sell_lim:
            self.sell()

In [19]:
class Position(SignalStrategy):
    def init(self):
        global df
        super().init()  
        self.set_signal(self.data.Position)

In [20]:
DATASET['Predicted_Class'].value_counts()

-2.0    467
 2.0    362
Name: Predicted_Class, dtype: int64

In [21]:
DATASET['Real_Class'].value_counts()

 0.0    201
-1.0    195
 1.0    186
-2.0    124
 2.0    123
Name: Real_Class, dtype: int64

In [22]:
STRATEGY_NAME = limits
print(STRATEGY_NAME.__name__)

limits


In [23]:
bt = Backtest(DATASET, STRATEGY_NAME, cash=CASH, commission=COMMISSION)#trade_on_close=False
bt.run()

Start                              0
End                              828
Duration                         828
Exposure [%]                 92.1498
Equity Final [$]             946.731
Equity Peak [$]              1054.41
Return [%]                  -5.32688
Buy & Hold Return [%]        5.26522
Max. Drawdown [%]           -10.4727
Avg. Drawdown [%]           -1.90889
Max. Drawdown Duration           728
Avg. Drawdown Duration       90.8889
# Trades                          75
Win Rate [%]                      48
Best Trade [%]               1.20754
Worst Trade [%]             -2.17541
Avg. Trade [%]            -0.0216111
Max. Trade Duration              232
Avg. Trade Duration          10.1733
Expectancy [%]               0.43877
SQN                        -0.320696
Sharpe Ratio              -0.0359319
Sortino Ratio             -0.0464281
Calmar Ratio             -0.00206356
_strategy                     limits
dtype: object

In [24]:
bt.plot()

In [25]:
data = {'Model_Name':[MODEL_NAME], 'PRED':[PRED], 'Strategy_Class':[STRATEGY_NAME.__name__], 'Equity_Final_$':[bt._results['Equity Final [$]']],
       'Equity_Peak_$':[bt._results['Equity Peak [$]']],'Return_%':[bt._results['Return [%]']],'#_Trades':[bt._results['# Trades']],
       'Win_Rate':[bt._results['Win Rate [%]']],'Commission':[COMMISSION],'Cash':[CASH]}
Results = pd.DataFrame(data)
print(Results)

                                          Model_Name  PRED Strategy_Class  \
0  5C_1D_45F_5137Size_LSTM_10Range_5Epochs_Class_...  True         limits   

   Equity_Final_$  Equity_Peak_$  Return_%  #_Trades  Win_Rate  Commission  \
0      946.731183    1054.411968 -5.326882      75.0      48.0      0.0001   

   Cash  
0  1000  


In [26]:
filename = 'BackTesting_Results.csv'
print(filename)
pwd = os.getcwd()
datasource_path = os.path.join(pwd, r'../../../Results/Backtesting/')
filename_path = os.path.join(pwd, r'../../../Results/Backtesting/' + filename)
print(filename_path)

BackTesting_Results.csv
/Users/marami3/Documents/ITESO/IDI4/forex_prediction/Jupyter Notebooks/IDI4/backtesting/../../../Results/Backtesting/BackTesting_Results.csv


In [27]:
#LOAD RESULTS .CSV
pwd = os.getcwd()
filenames = glob.glob(datasource_path + "/*.csv")
filenames.sort()
for file in filenames:
    print(os.path.basename(file))

BackTesting_Results.csv


In [29]:
Results_loaded = pd.read_csv(filenames[0], header=0)
#print(Results_loaded)

In [34]:
#print(Results)
Results_loaded = Results_loaded.append(Results,ignore_index=True)
print(Results_loaded[-1:])

                                           Model_Name  PRED Strategy_Class  \
15  5C_1D_45F_5137Size_LSTM_10Range_5Epochs_Class_...  True         limits   

    Equity_Final_$  Equity_Peak_$  Return_%  #_Trades  Win_Rate  Commission  \
15      946.731183    1054.411968 -5.326882      75.0      48.0      0.0001   

    Cash  
15  1000  


In [31]:
Results_loaded.to_csv(filename_path, header=True, index=False)